In [1]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (AutoTokenizer, AutoConfig,
    AutoModelForSequenceClassification, Trainer, TrainingArguments,
    DataCollatorWithPadding)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import torch
import numpy as np
import evaluate

In [2]:
model_checkpoint = "distilbert-base-uncased"

id2label = {0: 'Negative', 1: 'Positive'}
label2id = {'Negative': 0, 'Positive': 1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 2, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
dataset = load_dataset("shawhin/imdb-truncated")
print(dataset)

README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

(…)-00000-of-00001-5a744bf76a1d84b2.parquet:   0%|          | 0.00/836k [00:00<?, ?B/s]

(…)-00000-of-00001-a3a52fabb70c739f.parquet:   0%|          | 0.00/853k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})


In [4]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# create tokenize function
def tokenize_function(examples):
    text = examples["text"]

    # tokenize and truncate the text if it is too long
    tokenizer.truncation_side = 'left'
    tokenized_inputs = tokenizer(text, return_tensors='np', truncation=True, max_length=512)

    return tokenized_inputs

# add pad token if None exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# tokenize the training and validation sets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [5]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# define evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    # Use argmax for other metrics that require class predictions
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels),
        "precision": precision.compute(predictions=predictions, references=labels),
        "recall": recall.compute(predictions=predictions, references=labels),
        "f1": f1.compute(predictions=predictions, references=labels)
    }

In [7]:
# Untrained model predictions/performance

text_list = ["It was good.", "Not a fan, don't recommend.",
             "Better than the first one.", "This is not worth watching even once.",
             "This one is a pass."]

print("Untrained model predictions:")
print("---------------------------")

# Get the device of the model
device = next(model.parameters()).device

for text in text_list:
    # tokenize the text and move to the same device as the model
    inputs = tokenizer.encode(text, return_tensors='pt').to(device)
    # compute logits
    logits =  model(inputs).logits
    # convert logits to labels
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
---------------------------
It was good. - Negative
Not a fan, don't recommend. - Negative
Better than the first one. - Negative
This is not worth watching even once. - Negative
This one is a pass. - Negative


In [8]:
peft_config = LoraConfig(task_type='SEQ_CLS', # sequence classification
                         r = 4, # intrinsic rank of trainable weight matrix
                         lora_alpha=32,  # this is like learning rate
                         lora_dropout=0.01, # probability of dropout
                         target_modules=['q_lin'])  # we apply lora to query layer

In [9]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [10]:
# hyperparameters
lr = 1e-4
batch_size = 8
num_epochs = 10

# define training arguments
training_args = TrainingArguments(output_dir=model_checkpoint + '-lora-text-classification',
                learning_rate=lr,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=num_epochs,
                weight_decay=0.01,
                eval_strategy='epoch',
                save_strategy='epoch',
                load_best_model_at_end=True,
                report_to="none")

In [11]:
# create trainer object
trainer = Trainer(model=model, args=training_args, data_collator=data_collator,
                  train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['validation'],
                  tokenizer = tokenizer, compute_metrics=compute_metrics)

trainer.train()

/tmp/ipython-input-11-1062403234.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args, data_collator=data_collator,
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.397471,{'accuracy': 0.869},{'precision': 0.8448598130841122},{'recall': 0.904},{'f1': 0.8734299516908213}
2,No log,0.266523,{'accuracy': 0.893},{'precision': 0.9337748344370861},{'recall': 0.846},{'f1': 0.887722980062959}
3,No log,0.260335,{'accuracy': 0.898},{'precision': 0.9180672268907563},{'recall': 0.874},{'f1': 0.8954918032786885}
4,0.376900,0.262262,{'accuracy': 0.904},{'precision': 0.9105691056910569},{'recall': 0.896},{'f1': 0.9032258064516129}
5,0.376900,0.288784,{'accuracy': 0.9},{'precision': 0.9310344827586207},{'recall': 0.864},{'f1': 0.8962655601659751}
6,0.376900,0.275781,{'accuracy': 0.904},{'precision': 0.9105691056910569},{'recall': 0.896},{'f1': 0.9032258064516129}
7,0.376900,0.282606,{'accuracy': 0.907},{'precision': 0.9213250517598344},{'recall': 0.89},{'f1': 0.9053916581892166}
8,0.231600,0.287620,{'accuracy': 0.908},{'precision': 0.908},{'recall': 0.908},{'f1': 0.908}
9,0.231600,0.293282,{'accuracy': 0.908},{'precision': 0.9146341463414634},{'recall': 0.9},{'f1': 0.907258064516129}
10,0.231600,0.294446,{'accuracy': 0.908},{'precision': 0.9163265306122449},{'recall': 0.898},{'f1': 0.907070707070707}


TrainOutput(global_step=1250, training_loss=0.2833731811523437, metrics={'train_runtime': 502.9787, 'train_samples_per_second': 19.882, 'train_steps_per_second': 2.485, 'total_flos': 1253694805157184.0, 'train_loss': 0.2833731811523437, 'epoch': 10.0})

In [12]:
print("Trained model predictions:")
print("-------------------------")

device = next(model.parameters()).device

for text in text_list:
    # tokenize the text
    inputs = tokenizer.encode(text, return_tensors='pt').to(device)
    # compute logits
    logits =  model(inputs).logits
    # convert logits to labels
    predictions = torch.argmax(logits, 1)

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
-------------------------
It was good. - Positive
Not a fan, don't recommend. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Negative
